Working with a real world data-set using SQL and Python

Introduction

- Understand the dataset for Chicago Public School level performance
- Store the dataset in an Db2 database on IBM Cloud instance
- Retrieve metadata about tables and columns and query data from mixed case columns
- Solve example problems to practice your SQL skills including using built-in database functions

Chicago Public Schools - Progress Report Cards (2011-2012)

The city of Chicago released a dataset showing all school level performance data used to create School Report Cards for the 2011-2012 school year. The dataset is available from the Chicago Data Portal: https://data.cityofchicago.org/Education/Chicago-Public-Schools-Progress-Report-Cards-2011-/9xs2-f89t


This dataset includes a large number of metrics. Start by familiarizing yourself with the types of metrics in the database: https://data.cityofchicago.org/api/assets/AAD41A13-BE8A-4E67-B1F5-86E711E09D5F?download=true


NOTE: Do not download the dataset directly from City of Chicago portal. Instead download a more database friendly version from the link below. Now download a static copy of this database and review some of its contents: https://ibm.box.com/shared/static/f9gjvj1gjmxxzycdhplzt01qtz0s7ew7.csv

Store the dataset in a Table
In many cases the dataset to be analyzed is available as a .CSV (comma separated values) file, perhaps on the internet. To analyze the data using SQL, it first needs to be stored in the database.

On Db2:

Open the Db2 console, open the LOAD tool, Select / Drag the .CSV file for the CHICAGO PUBLIC SCHOOLS dataset and load the dataset into a new table called SCHOOLS.

Connect to the database

In [ ]:
#load the ipython-sql extension and establish a connection with the database
%load_ext sql

In [ ]:
# Enter the connection string for your Db2 on Cloud database instance below
# %sql ibm_db_sa://my-username:my-password@my-hostname:my-port/my-db-name

%sql ibm_db_sa://xxxxxxxxxx

Query the database system catalog to retrieve table metadata

In [ ]:
# type in your query to retrieve list of all tables in the database for your db2 schema (username)

%sql select * from SYSCAT.TABLES where TABNAME = 'FQS42390.CHICAGO_SOCIOECONOMIC_DATA'


<!-- Solution:
%sql select TABSCHEMA, TABNAME, CREATE_TIME from SYSCAT.TABLES where TABSCHEMA='YOUR-DB2-USERNAME'

or, you can retrieve list of all tables where the schema name is not one of the system created ones:

%sql select TABSCHEMA, TABNAME, CREATE_TIME from SYSCAT.TABLES \
      where TABSCHEMA not in ('SYSIBM', 'SYSCAT', 'SYSSTAT', 'SYSIBMADM', 'SYSTOOLS', 'SYSPUBLIC')
      
or, just query for a specifc table that you want to verify exists in the database
%sql select * from SYSCAT.TABLES where TABNAME = 'SCHOOLS'
-->

Query the database system catalog to retrieve column metadata

In [ ]:
# type in your query to retrieve the number of columns in the SCHOOLS table
%sql select count(*) from SYSCAT.COLUMNS where TABNAME = 'FQS42390.CHICAGO_SOCIOECONOMIC_DATA'

<!-- Solution:
%sql select count(*) from SYSCAT.COLUMNS where TABNAME = 'SCHOOLS'
-->

In [ ]:
# type in your query to retrieve all column names in the SCHOOLS table along with their datatypes and length
%sql select COLNAME, TYPENAME, LENGTH from SYSCAT.COLUMNS where TABNAME = 'FQS42390.CHICAGO_SOCIOECONOMIC_DATA'

%sql select distinct(NAME), COLTYPE, LENGTH from SYSIBM.SYSCOLUMNS where TBNAME = 'FQS42390.CHICAGO_SOCIOECONOMIC_DATA'


<!-- Solution:
%sql select COLNAME, TYPENAME, LENGTH from SYSCAT.COLUMNS where TABNAME = 'SCHOOLS'
or
%sql select distinct(NAME), COLTYPE, LENGTH from SYSIBM.SYSCOLUMNS where TBNAME = 'SCHOOLS'
-->

Analysis

In [ ]:
# 1.How many Elementary Schools are in the dataset?
%sql select count(*) from SCHOOLS where "Elementary, Middle, or High School" = 'ES'

# Correct answer: 462

In [ ]:
# 2.What is the highest Safety Score?
%sql select MAX(Safety_Score) AS MAX_SAFETY_SCORE from SCHOOLS

# Correct answer: 99

In [ ]:
# 3. Which schools have highest Safety Score?

# In the previous problem we found out that the highest Safety Score is 99, so we can use that as an input in the where clause:

%sql select Name_of_School, Safety_Score from SCHOOLS where Safety_Score = 99

# or, a better way:

%sql select Name_of_School, Safety_Score from FQS42390.CHICAGO_SOCIOECONOMIC_DATA
 where \
  Safety_Score= (select MAX(Safety_Score) from FQS42390.CHICAGO_SOCIOECONOMIC_DATA
)


# Correct answer: several schools with with Safety Score of 99.


In [ ]:
# 4.What are the top 10 schools with the highest "Average Student Attendance"?
%sql select Name_of_School, Average_Student_Attendance from SCHOOLS \
    order by Average_Student_Attendance desc nulls last limit 10 

In [ ]:
# 5. Retrieve the list of 5 Schools with the lowest Average Student Attendance sorted in ascending order based on attendance
%sql SELECT Name_of_School, Average_Student_Attendance  \
     from SCHOOLS \
     order by Average_Student_Attendance \
     fetch first 5 rows only

In [ ]:
# 6. Now remove the '%' sign from the above result set for Average Student Attendance column
%sql SELECT Name_of_School, REPLACE(Average_Student_Attendance, '%', '') \
     from SCHOOLS \
     order by Average_Student_Attendance \
     fetch first 5 rows only


In [ ]:
# 7. Which Schools have Average Student Attendance lower than 70%?
%sql SELECT Name_of_School, Average_Student_Attendance  \
     from SCHOOLS \
     where CAST ( REPLACE(Average_Student_Attendance, '%', '') AS DOUBLE ) < 70 \
     order by Average_Student_Attendance
     
or,

%sql SELECT Name_of_School, Average_Student_Attendance  \
     from SCHOOLS \
     where DECIMAL ( REPLACE(Average_Student_Attendance, '%', '') ) < 70 \
     order by Average_Student_Attendance


In [ ]:
# 8. Get the total College Enrollment for each Community Area
%sql select Community_Area_Name, sum(College_Enrollment) AS TOTAL_ENROLLMENT \
   from SCHOOLS \
   group by Community_Area_Name 


In [ ]:
# 9.Get the 5 Community Areas with the least total College Enrollment sorted in ascending order
%sql select Community_Area_Name, sum(College_Enrollment) AS TOTAL_ENROLLMENT \
   from SCHOOLS \
   group by Community_Area_Name \
   order by TOTAL_ENROLLMENT asc \
   fetch first 5 rows only


In [ ]:
# 10.Get the hardship index for the community area which has College Enrollment of 4638
#NOTE: For this solution to work the CHICAGO_SOCIOECONOMIC_DATA table 
    

%%sql 
select hardship_index 
   from chicago_socioeconomic_data CD, schools CPS 
   where CD.ca = CPS.community_area_number 
      and college_enrollment = 4368

In [ ]:
# 11.Get the hardship index for the community area which has the highest value for College Enrollment
#NOTE: For this solution to work the CHICAGO_SOCIOECONOMIC_DATA table 


%sql select ca, community_area_name, hardship_index from chicago_socioeconomic_data \
   where ca in \
   ( select community_area_number from schools order by college_enrollment desc limit 1 )

In [ ]:
ibm_db.close(conn)